In [1]:
import os
%pwd

'e:\\Code\\NLP_Text_Summarize_Project\\research'

In [2]:
os.chdir("../")
%pwd

'e:\\Code\\NLP_Text_Summarize_Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from src.Text_Summarize.constants import *
from src.Text_Summarize.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [6]:
from src.Text_Summarize import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

e:\C.TSUBASA\anaconda\envs\potatoes\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-19 22:24:27,856: INFO: config: TensorFlow version 2.16.1 available.]


In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch: dict) -> dict:
        """
        Convert a batch of examples to features using the tokenizer.

        Args:
            example_batch (dict): A batch of examples with 'dialogue' and'summary' keys.

        Returns:
            dict: A dictionary with 'input_ids', 'attention_mask', and 'labels' keys.
        """
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def process_and_save_data(self) -> None:
        """
        Load data from disk, convert examples to features, and save the result to disk.
        """
        try:
            dataset_corpus = load_from_disk(self.config.data_path)
            dataset_corpus_pt = dataset_corpus.map(self.convert_examples_to_features, batched=True)
            dataset_corpus_pt.save_to_disk(os.path.join(self.config.root_dir, "corpus_dataset"))
        except Exception as e:
            print(f"Error processing data: {e}")

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.process_and_save_data()
except ValueError as e:
    raise ValueError("Error instantiating tokenizer. Make sure sentencepiece is installed.") from e


[2024-05-19 22:24:38,765: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2024-05-19 22:24:38,769: INFO: common: yaml file: params.yaml loaded succesfully]
[2024-05-19 22:24:38,771: INFO: common: created directory at: artifacts]
[2024-05-19 22:24:38,773: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]e:\C.TSUBASA\anaconda\envs\potatoes\Lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 35909.16 examples/s]
